In [3]:
#pharmacovigilance signal detection, openfda api, pytorch, prophet, time series 
#jack baxter, 10/2025

In [ ]:
#load packs
import requests
import pandas as pd
import plotly.graph_objects as go
from prophet import Prophet
from darts import TimeSeries
from darts.models import NBEATSModel
from pytorch_forecasting import TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import QuantileLoss
import torch
import warnings
warnings.filterwarnings("ignore")

/opt/miniconda3/envs/pharmaco/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.


In [15]:
drugchoice = input('Please enter the name of the drug to investigate: ')

In [16]:
def get_fda_data(drugchoice):
    url = "https://api.fda.gov/drug/event.json"
    query = f'patient.drug.medicinalproduct:"{drugchoice}"+AND+receivedate:[20140101+TO+20251231]'
    resp = requests.get(url, params={"search": query, "count": "receivedate"}).json()
    if 'results' not in url:
        print(f'No data found for {drugchoice}. Please check the drug name and try again.')
        return pd.DataFrame()
    df = pd.DataFrame(resp['results'])
    df['receivedate'] = pd.to_datetime(df['receivedate'].astype(str), format='%Y%m%d', errors='coerce')
    df = df.dropna(subset=['receivedate'])
    df = df.set_index('receivedate').resample('Q').size().reset_index()
    df.columns = ['ds', 'y']
    df['ds'] = pd.date_range(start=df['ds'].min(), end=df['ds'].max(), freq='Q')
    df = df.set_index('ds').reindex(df['ds']).fillna(0).reset_index()
    print(f"Loaded {len(df)} quarters for {drugchoice} (total reports: {df['y'].sum()})")
    return df

In [17]:
df = get_fda_data(drugchoice)
df.head()

No data found for semaglutide. Please check the drug name and try again.


""
